#### In this question we analyze the data file **Fertility.csv** which is available in the homework folder, and try to build a model for fertility. In this dataset Fertility, the first column, is the response variable, and the other variables are potential predictors. We will use several different statistical modeling techniques. The data set contains 47 rows (samples), split the data into training and test sets. Set the first 30 rows to training samples and the rows 31 through 47 as the test samples.

In [17]:
import numpy as np
import pandas as pd
from ISLP.models import ModelSpec as MS
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import sklearn.linear_model as skl
from sklearn.linear_model import Ridge, Lasso, LinearRegression, RidgeCV, LassoCV
from sklearn.metrics import mean_squared_error

In [9]:
Data = pd.read_csv('Fertility.csv')
# print(Data)
train = (Data.index < 30)
X = MS(Data.columns.drop(['Fertility'])).fit_transform(Data)
Y = Data['Fertility']
print(X.shape)
y_train, X_train = Y.loc[train] , X.loc[train]
y_test, X_test = Y.loc[~train] , X.loc[~train]

(47, 6)


#### (a) Fit a linear model on the training set, and report the test error (MSE) obtained.

In [10]:
LinearModel = LinearRegression()
LinearModel.fit(X_train, y_train)
y_pred_linear = LinearModel.predict(X_test)
mse_linear = mean_squared_error(y_test, y_pred_linear)
# print(pd.DataFrame({'y':y_test, 'pred':pred_linear}))
print('MSE Linear:', mse_linear)

MSE Linear: 183.72179150160574


#### (b) Fit a Ridge regression model on the training set, with λ chosen by cross-validation on a dense grid similar to the example solved in the class. Report the test error obtained.

In [11]:
lambdas = np.logspace(-12, 12, 100)  # Define a range of lambda values
# print(lambdas)
ridge_model = RidgeCV(alphas=lambdas, store_cv_values=True).fit(X_train, y_train)
y_pred_ridge = ridge_model.predict(X_test)
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
print('Best lambda for Ridge:', ridge_model.alpha_)
print('MSE Ridge: ', mse_ridge)

Best lambda for Ridge: 200.92330025650458
MSE Ridge:  190.8866277059537


In [20]:
# print(lambdas[39], np.linalg.norm(beta_hat1))
# print(lambdas[59], np.linalg.norm(beta_hat2))
# ridge = skl.ElasticNet(alpha=lambdas[39], l1_ratio=0)
scaler = StandardScaler(with_mean=True,  with_std=True)
# pipe = Pipeline(steps=[('scaler', scaler), ('ridge', ridge)]) #  why using Pipeline?
# pipe.fit(X, Y)

# ridge = Ridge()
K = 5
# kfold = KFold(K, random_state=0, shuffle=True)
# param_grid = {'ridge__alpha': lambdas}
# grid = GridSearchCV(pipe, param_grid, cv=kfold, scoring='neg_mean_squared_error')
# grid.fit(X, Y)
# grid.best_params_['ridge__alpha']
# print(grid.best_estimator_)

#### (c) Fit a LASSO model on the training set, with λ chosen by cross-validation on a dense grid similar to the example solved in the class. Report the test error obtained, along with the number of non-zero coefficient estimates.

In [13]:
# 5-Fold
lasso_model = LassoCV(alphas=lambdas, cv=5).fit(X_train, y_train)

y_pred_lasso = lasso_model.predict(X_test)

mse_lasso = mean_squared_error(y_test, y_pred_lasso)
# print(lasso_model.coef_)
non_zero_coefficients = np.sum(lasso_model.coef_ != 0)

print('Best lambda LASSO:', lasso_model.alpha_)
print('MSE LASSO:', mse_lasso)
print('How many non-zero coefficient estimates: ', non_zero_coefficients)

Best lambda LASSO: 7.05480231071866
MSE LASSO: 262.33615824293395
How many non-zero coefficient estimates:  2


In [15]:
# Xs = X - np.mean(X, axis = 0)[None,:]
# Xs = X - np.mean(X, axis=0, keepdims=True)
# kfold = skm.KFold(5, random_state=0, shuffle=True)
# scaler = StandardScaler(with_mean=True,  with_std=True)

# lassoCV = skl.ElasticNetCV(n_alphas=100, l1_ratio=1, cv=kfold)
# pipeCV = Pipeline(steps=[('scaler', scaler), ('lasso', lassoCV)])
# pipeCV.fit(X, Y)
# tuned_lasso = pipeCV.named_steps['lasso']
# print(tuned_lasso.alpha_) #printing the lambda that yields the smallest CV error


# lambdas, soln_array = skl.Lasso.path(Xs, Y, l1_ratio=1, n_alphas=100)[:2]
# soln_path = pd.DataFrame(soln_array.T, columns=X.columns, index=-np.log(lambdas))

#### (d) Compare the results of (a), (b), and (c). Which one seems to outperform the others for this specific setup?

#### In this case, the Linear Model got the lowest MSE compared to Ridge and LASSO. The Ridge model is pretty close but got a slightly higher MSE. The LASSO has 2 non-dropped features and the highest MSE of the three models, which means that too many constraints or regularization led to underfitting.

In [24]:
ridgeCV = skl.ElasticNetCV(alphas=lambdas, 
                           l1_ratio=0,
                           # tol = 0.01,
                           cv=KFold(5, random_state=0, shuffle=True))
pipeCV = Pipeline(steps=[('scaler', scaler),
                         ('ridge', ridgeCV)])
pipeCV.fit(X_train, y_train)
# print('lambda:', ridgeCV.alpha_)
# print(X)
# print('coef:', ridgeCV.coef_)
# print('intercept:', ridgeCV.intercept_)
# print('mse_ridge:', np.mean(ridgeCV.mse_path_))

/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:617: UserWarning: Coordinate descent without L1 regularization may lead to unexpected results and is discouraged. Set l1_ratio > 0 to add L1 regularization.
  model = cd_fast.enet_coordinate_descent_gram(
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1359.2566666644716, tolerance: 0.2718513333333334
  model = cd_fast.enet_coordinate_descent_gram(
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:617: UserWarning: Coordinate descent without L1 regularization may lead to unexpected results and is discouraged. Set l1_ratio > 0 to add L1 regularization.
  model = cd_fast.enet_coordinate_descent_gram(
/usr/local/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.

Pipeline(steps=[('scaler', StandardScaler()),
                ('ridge',
                 ElasticNetCV(alphas=array([1.00000000e-12, 1.74752840e-12, 3.05385551e-12, 5.33669923e-12,
       9.32603347e-12, 1.62975083e-11, 2.84803587e-11, 4.97702356e-11,
       8.69749003e-11, 1.51991108e-10, 2.65608778e-10, 4.64158883e-10,
       8.11130831e-10, 1.41747416e-09, 2.47707636e-09, 4.32876128e-09,
       7.56463328e-09, 1.32194115e-08,...
       2.47707636e+07, 4.32876128e+07, 7.56463328e+07, 1.32194115e+08,
       2.31012970e+08, 4.03701726e+08, 7.05480231e+08, 1.23284674e+09,
       2.15443469e+09, 3.76493581e+09, 6.57933225e+09, 1.14975700e+10,
       2.00923300e+10, 3.51119173e+10, 6.13590727e+10, 1.07226722e+11,
       1.87381742e+11, 3.27454916e+11, 5.72236766e+11, 1.00000000e+12]),
                              cv=KFold(n_splits=5, random_state=0, shuffle=True),
                              l1_ratio=0))])